In [1]:
import sys
print(sys.version)
print("executable: ")
print(sys.executable)
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets as tfds
print(tfds.__version__)
#print("TFDS version:", tfds.__version__)
from tensorflow.keras import layers, models
import numpy as np
#import tensorflow.keras as keras

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]
executable: 
/usr/bin/python


2025-01-12 22:36:10.966354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 22:36:11.154682: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 22:36:12.192400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-12 22:36:12.192516

2.11.0
4.9.2


In [2]:
#loading mnist data from tensorflow
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
#mnist = tfds.load(name='mnist', split='train', as_supervised=True)
# Returns both train and test split separately
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    image = image
    #image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1] # i believe this is not in the original code
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image)
mnist_test = mnist_test.map(preprocess_image)
mnist_train_atn = mnist_train.batch(100)
mnist_test_atn = mnist_test.batch(100) #not used now since test is done in other code still

2025-01-12 22:36:22.301452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13232 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:1b:00.0, compute capability: 8.6
2025-01-12 22:36:22.303870: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13232 MB memory:  -> device: 1, name: NVIDIA A16, pci bus id: 0000:1c:00.0, compute capability: 8.6
2025-01-12 22:36:22.306071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13232 MB memory:  -> device: 2, name: NVIDIA A16, pci bus id: 0000:1d:00.0, compute capability: 8.6
2025-01-12 22:36:22.308145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13232 MB memory:  -> device: 3, name: NVIDIA A16, pci bus id: 0000:1e:00.0, compute capability: 8.6


In [3]:
#defining the network
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [4]:
#initialising the weights with random values
# NOT USED IN THIS CODE WITH KERAS IMPLEMENTATION
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [5]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [6]:
#function for forward prop
#This function is edited now with keras functionality. weights are now handled internally but can still be accessed with get_weights method
def cnn(x):
    # Define the layers using Keras API
    #x = layers.Reshape((input_height, input_width, input_channels))(x)
    
    conv1 = layers.Conv2D(n_conv1, kernel_size=(conv1_k, conv1_k), strides=stride_conv1, padding="same", activation="relu")(x)
    conv1_pool = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2, kernel_size=(conv2_k, conv2_k), strides=stride_conv2, padding="same", activation="relu")(conv1_pool)
    conv2_pool = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    # Flatten the output from convolutional layers
    flatten = layers.Flatten()(conv2_pool)

    # Fully connected layer
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # Dropout
    dropout = layers.Dropout(0.2)(hidden)

    # Output layer (logits)
    output = layers.Dense(n_out, activation = 'softmax')(dropout)
    
    return output
    
    #x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    #conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    #conv1_pool = maxpooling(conv1, max_pool1_k)
    
    #conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    #conv2_pool = maxpooling(conv2, max_pool2_k)
    
    #hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    #hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    #hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    #hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    #output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo']) #no softmax activation function since the loss function handles it already
    #return output

In [7]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
pred = cnn(x)

In [8]:
model = models.Model(inputs=[x], outputs=pred)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train #it will be given in batches by the create_partly_adv_data method
test_data = mnist_test.batch(100)
# Get a single batch from the test data (a batch is a tuple of (image, label))
#single_batch = next(iter(test_data))

# single_batch is a tuple, where single_batch[0] contains the image and single_batch[1] contains the label
#image, label = single_batch

# You can print or inspect the first image and label from the batch
#print(image[0].numpy())  # The first image in the batch
#print(label[0].numpy())  # The label for the first image


In [9]:
# @ layers with 2 deconvolution layers
def build_atn(input_shape):
    model = tf.keras.Sequential([
        # First convolutional layers to capture features
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),  # Downsample (14, 14, 64)

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),  # Downsample (7, 7, 128)

        # Fully connected layer to integrate features
        layers.Flatten(),  # Flatten to shape (7 * 7 * 128 = 6272)
        layers.Dense(6272, activation='relu'),  # Match the reshape target

        # Deconvolution layers to reconstruct adversarial examples
        layers.Reshape((7, 7, 128)),  # Reshape for deconvolution
        layers.Conv2DTranspose(128, (3, 3), activation='relu', strides=2, padding='same'),  # (14, 14, 128)
        layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same'),  # (28, 28, 64)
        
        layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'),  # Output layer (28, 28, 1)
        layers.Lambda(lambda x: x * 255)  # Scale to [0, 255]
    ])
    return model

# Build the ATN
input_shape = (28, 28, 1)  # MNIST dataset shape
atn = build_atn(input_shape)

In [10]:
# Define input loss and output loss function
def input_loss(x, x_prime):
    return tf.reduce_mean(tf.square(tf.cast(x, tf.float32) - x_prime))  # Cast x to float32  # L2 loss

def output_loss(y_pred, y_target):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_target, y_pred)) # crossentropy


In [11]:
# Combine the models
input_img = tf.keras.Input(shape=input_shape)
x_prime = atn(input_img)
y_pred = model(x_prime)

combined_model = tf.keras.Model(inputs=input_img, outputs=[x_prime, y_pred])

In [12]:
def get_second_highest_target(y_pred_original):
    # Sort predictions to find the highest and second-highest indices
    top_2_indices = tf.argsort(y_pred_original, direction='DESCENDING', axis=1)[:, :2]
    second_highest_indices = top_2_indices[:, 1]

    # Create one-hot encoding for the second-highest class
    y_target = tf.one_hot(second_highest_indices, depth=y_pred_original.shape[1])
    return y_target

In [13]:
# Define training procedure
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
beta = 0.1  # Weight for input-space loss

@tf.function
def atn_train_step(x, y_original):
    with tf.GradientTape() as tape:
        # Generate adversarial examples
        x_prime = atn(x)

        # Get predictions for original and adversarial inputs
        y_pred_original = model(x, training=False)
        y_pred = model(x_prime, training=False)

        # Compute the second-highest target dynamically
        y_target = get_second_highest_target(y_pred_original)

        # Compute input loss and output loss separately
        l_x = input_loss(x, x_prime)
        l_y = output_loss(y_pred, y_target)

        # Compute the combined loss
        loss = beta * l_x + l_y

    # Compute and apply gradients
    gradients = tape.gradient(loss, atn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, atn.trainable_variables))

    # Return total loss, input loss, and output loss
    return loss, l_x, l_y

In [14]:
from tqdm import tqdm
import time

def train_atn():

    # Training loop
    epochs = 5 #for experiment reasons, changed from 50 to 5 now
    batch_size = 32
    beta = 0.01  # Weight for the input-space loss
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    print(f"{'Epoch':<8}{'Total Loss':<12}{'Scaled Input Loss':<12}{'Output Loss':<12}{'Time (s)':<10}")
    print("-" * 50)

    for epoch in range(epochs):
        start_time = time.time()
        epoch_loss = 0
        epoch_input_loss = 0
        epoch_output_loss = 0
        batch_count = 0

        # Add progress bar for batches
        for x_batch, y_original_batch in tqdm(mnist_train_atn, desc=f"Epoch {epoch + 1}/{epochs}"):
            loss, l_x, l_y = atn_train_step(x_batch, y_original_batch)  # Get all losses
            epoch_loss += loss.numpy()
            epoch_input_loss += l_x.numpy()*beta
            epoch_output_loss += l_y.numpy()
            batch_count += 1

        # Compute average losses for the epoch
        avg_loss = epoch_loss / batch_count
        avg_input_loss = epoch_input_loss / batch_count
        avg_output_loss = epoch_output_loss / batch_count
        elapsed_time = time.time() - start_time

        # Print epoch summary
        print(f"{epoch + 1:<8}{avg_loss:<12.4f}{avg_input_loss:<12.4f}{avg_output_loss:<12.4f}{elapsed_time:<10.2f}")


In [15]:
# Added this cell for adversarial training
#keras.config.enable_unsafe_deserialization()
#atn = tf.keras.models.load_model('jurjen_adversarial_transformation_network_beta-0.01_epochs-50_conv2d_transpose.keras')

def create_partly_adv_data(atn, dataset, indices):
    mnist_train_images = np.array([x.numpy() for x, y in dataset])
    mnist_train_labels = np.array([y.numpy() for x, y in dataset])
    #random should ensure every epoch new images are used to generate adversarial samples
    adversarial_samples = atn.predict(mnist_train_images[indices]) #get list of x prime's
    print(f"How many adversarial samples: {len(adversarial_samples)}")
    mnist_train_images[indices] = adversarial_samples
    #recreate the tf.data.Dataset with the modified data
    updated_mnist_train = tf.data.Dataset.from_tensor_slices((mnist_train_images, mnist_train_labels)).batch(100) #return it in batches of 100
    return updated_mnist_train, adversarial_samples, mnist_train_labels[indices]

In [16]:
def cnn_train_step(batch_data, batch_labels, adversarial_samples, adversarial_labels, batchnumber):
    global train_data_with_adv  # Allow modifications to train_data_with_adv
    global train_data_iterator
    # adv_true_labels is not a global variable since it is not changed, it just represents a subset of the train labels
    
    # Train the model on the current batch
    model.train_on_batch(batch_data, batch_labels)
    
    # calculate accuracy on all adversarial examples of this epoch after each 50 batches
    if ((batch_number + 1) % 50 == 0):
         # Evaluate the model on all adversarial examples for the epoch
        adv_preds_classes = np.argmax(model.predict(adversarial_samples), axis=1)
        adv_true_classes = np.argmax(adversarial_labels, axis=1)
        
        # Compute accuracy on all adversarial examples
        adv_accuracy = np.mean(adv_preds_classes == adv_true_classes) * 100
        print(f"Accuracy on all adversarial examples after batch {batchnumber + 1}: {adv_accuracy:.2f}%")
        if adv_accuracy > 50:
            print("too high accuracy on adversarial samples, retraining atn")
            train_atn()
            #replace the existing adversarial examples with new adversarial examples. 
            train_data_with_adv, adversarial_examples, adv_true_labels = create_partly_adv_data(atn, train_data, indices) #adv_true_labels here is not used outside this method as it is not changed by the method
            train_data_iterator = iter(train_data_with_adv) #iterator is regenerated, initial one is assigned to old training data
            for b in range(batch_number + 1):  # for the newly generated iterator skip up to the current batch number to continue where we left off.
                next(train_data_iterator)
            print("ATN retrained, adversarial examples and train_data_with_adv updated.")

            
        #adv_indices = np.isin(batch_data.numpy(), adversarial_examples).nonzero()[0] #find the examples in the batch which are in the set adversarial examples 
        #if len(adv_indices) > 0:
        #    adv_batch = batch_data.numpy()[adv_indices]
        #    adv_batch_labels = adversarial_labels[adv_indices]
        
            # Get model predictions on adversarial samples
            #adv_preds = model.predict(adv_batch) #perhaps later useful if wanna work with softmax outputs
        #    adv_preds_classes = np.argmax((model.predict(adv_batch)), axis=1)
        #    adv_true_classes = np.argmax(adv_batch_labels, axis=1)
        
            # Compute accuracy on adversarial examples
        #    adv_accuracy = np.mean(adv_preds_classes == adv_true_classes) * 100
        #    print(f"Accuracy on adversarial examples in this batch: {adv_accuracy:.2f}%")
        #else:
        #    print("No adversarial examples in this batch.")

In [17]:
global adversarial_examples

#first we do the first training of the atn
train_atn()

fraction = 0.5
num_epochs = 2 #changed from 25 to 2 for experimentation reasons now.
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    #determine the indices for adversarial samples
    num_samples = len(mnist_train)
    num_adversarial = int(fraction * num_samples)
    indices = np.random.choice(num_samples, num_adversarial, replace=False)
    #create initial subset of adversarial examples. This is made explictly mutable as it can be changed by cnn_train_step after retraining the ATN
    train_data_with_adv, adversarial_examples, adv_true_labels = create_partly_adv_data(atn, train_data, indices)  #create adversarial samples per epoch
    #adversarial_examples = np.array(adversarial_examples)  # makes copy, Ensures mutability, adv_true_labels does not have to be mutable, since the ATN doesnt change this.
    
    train_data_iterator = iter(train_data_with_adv) # Create an iterator from the dataset
    for batch_number in range(len(train_data_with_adv)): #loop over the number of batches not the data itself as it may change during the loop
        print(f"batch : {batch_number + 1}")
        try:
            # Get the next batch
            x, labels = next(train_data_iterator)
            cnn_train_step(x, labels, adversarial_examples, adv_true_labels, batch_number) #adversarial_examples and adv true labels is only for checking accuracy. Need to modify train_data_with_adv
        
        except StopIteration:
            break # In case the iterator runs out of data
            
    #do not loop over something that is changing
    #for batch_number, (x, labels) in enumerate(train_data_with_adv):  # Loop over batches, train_data_with_adv is already batched
    #    cnn_train_step(x, labels, adversarial_examples, adv_true_labels, batch_number) #adversarial_examples and adv true labels is only for checking accuracy. Need to modify train_data_with_adv
    
    
    
    #model.fit(train_data_with_adv, epochs=1)
    
    #check per epoch how well the atn can fool the model
    adv_preds_classes = np.argmax(model.predict(adversarial_examples), axis=1)  # Get predicted classes
    adv_true_classes = np.argmax(adv_true_labels, axis=1)  # Get true classes #simply comparing accuracy on how much classes are correct. perhaps using cross entropy loss threshold is better
    # Calculate accuracy on adversarial examples
    adv_accuracy = np.mean(adv_preds_classes == adv_true_classes) * 100 # [True, False, True, True, True], take the mean of that, x100 for percentage
    print(f"Target model accuracy on adversarial samples for epoch {epoch + 1}: {adv_accuracy:.2f}%")

Epoch   Total Loss  Scaled Input LossOutput Loss Time (s)  
--------------------------------------------------


Epoch 1/5:   0%|          | 0/600 [00:00<?, ?it/s]2025-01-12 22:37:04.067089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2025-01-12 22:37:05.549789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Epoch 1/5: 100%|██████████| 600/600 [00:26<00:00, 22.88it/s]


1       105.1939    10.0446     4.7480      26.24     


Epoch 2/5: 100%|██████████| 600/600 [00:20<00:00, 28.76it/s]


2       27.4511     2.3098      4.3531      20.87     


...

3       16.2531     0.6627      9.6265      20.26     


Epoch 4/5: 100%|██████████| 600/600 [00:20<00:00, 29.65it/s]


4       16.2772     0.6741      9.5357      20.24     


Epoch 5/5: 100%|██████████| 600/600 [00:19<00:00, 30.36it/s]


5       16.2397     0.6782      9.4572      19.77     
938/938 [==============================] - 4s 5ms/step
How many adversarial samples: 30000
ATN retrained, adversarial examples and train_data_with_adv updated.
938/938 [==============================] - 4s 4ms/step
Target model accuracy on adversarial samples for epoch 2: 98.32%


In [18]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

100/100 [==============================] - 2s 12ms/step - loss: 0.0536 - accuracy: 0.9850
Test accuracy: 0.9850000143051147
summary: 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                       

In [19]:
model.save('jurjen_adv_training_3_model.keras')

In [22]:
# Save the ATN with the generated name
atn.save("jurjen_ATN_adv_training_3_beta0.01_epochs5_conv2d_transpose.keras")